In [2]:
import os
os.chdir('/home/s3/hyeryung/mucoco')

import pandas as pd

# Debug Issue 1. Low Bertscore (negative) for CLSF_BV0 & EM_BV0
## Merge all results into one file.

In [17]:
# with open('data/Sentiment-and-Style-Transfer/data/yelp/sentiment.test.0') as f:
#     source_0 = [line.rstrip('\n') for line in f.readlines()]

In [78]:
ref_0 = pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.0',sep='\t',header=None,names=['text','ref'])

In [80]:
with open('lewis/model_outputs/yelp/pos.out', 'r') as f:
    lewis_0 = [line.rstrip('\n') for line in f.readlines()]
ref_0['lewis']=lewis_0

In [81]:
clsf_bv0 = pd.read_json('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-vtwtoeyi-negative-to-positive/outputs_epsilon0.75.txt',
                        lines=True)

## unravel data
clsf_bv0=clsf_bv0.explode('generations')
clsf_bv0['text']=clsf_bv0['generations'].apply(lambda x: x['text'])

ref_0['clsf_bv0']=clsf_bv0['text']

In [82]:
em_bv0 = pd.read_json('outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-dppnf7pu-negative-to-positive/outputs_epsilon0.75.txt', 
                      lines=True)

## unravel data
em_bv0=em_bv0.explode('generations')
em_bv0['text']=em_bv0['generations'].apply(lambda x: x['text'])

ref_0['em_bv0']=em_bv0['text']

In [83]:
lewis_bert = pd.read_csv('lewis/model_outputs/yelp/results-negative-to-positive.txt.bertscore')
ref_0['bert_lewis']=lewis_bert['bert_score']
ref_0['sbert_lewis']=lewis_bert['sbert_score']

In [84]:
lewis_cls = pd.read_csv('lewis/model_outputs/yelp/results-negative-to-positive.txt.cls')
ref_0['cls_lewis']=lewis_cls['sentiment_predictions']

In [85]:
clsf_bv0_bert = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.bertscore')
ref_0['bert_clsf_bv0']=clsf_bv0_bert['bert_score']
ref_0['sbert_clsf_bv0']=clsf_bv0_bert['sbert_score']

In [86]:
clsf_bv0_cls = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.cls')
ref_0['cls_clsf_bv0']=clsf_bv0_cls['sentiment_predictions']

In [87]:
em_bv0_bert = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.bertscore')
ref_0['bert_em_bv0']=em_bv0_bert['bert_score']
ref_0['sbert_em_bv0']=em_bv0_bert['sbert_score']

In [88]:
em_bv0_cls = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.cls')
ref_0['cls_em_bv0']=em_bv0_cls['sentiment_predictions']

In [94]:
ref_0.to_excel('new_module/compr/evaluate_lewis.xlsx')

## Found Issue 1. For EM_BV0, it seems that quite a lot of samples are '' -> which result in -4.925 in bert scores. 
Solution: rerun decoding so that regardless of initial text's satisfaction of constraint, locate&edit is conducted for all samples.

## Found Issue 2. For CLSF_BV0, strangely, the rows with -4.925 BertScore has string. Find out why. -> Found out that I reversed the pred_0 and pred_1 when evaluating clsf_bv0. 
Solution: rerun decoding with correct files.

In [12]:
import evaluate

In [5]:
ref_0=pd.read_excel('new_module/compr/evaluate_lewis.xlsx',index_col=0)

In [7]:
debug_ref_0 = ref_0.loc[ref_0['bert_clsf_bv0']<-3,:].copy()

In [8]:
debug_ref_0

,text,ref,lewis,clsf_bv0,em_bv0,bert_lewis,sbert_lewis,cls_lewis,bert_clsf_bv0,sbert_clsf_bv0,cls_clsf_bv0,bert_em_bv0,sbert_em_bv0,cls_em_bv0
13,said we could n't sit at the table if we were ...,they said we could sit at the table with no he...,we were so impressed we could n't sit at the t...,"""We couldn't be at the table if we weren't in ...",I think we have to sit at the table if we want...,0.318293,0.635969,1,-4.925075,-4.925075,1,0.336163,0.319579,0
18,the burgers were over cooked to the point the ...,the burgers were cooked perfectly and the meat...,the burgers were perfectly cooked to the point...,But they were so close to the point the game w...,the burgers have been cooked to the point the ...,0.748451,0.790539,1,-4.925075,-4.925075,1,0.588865,0.810976,0
56,lost a long time customer !,gained a long time customer !,i will be a long time customer !,"Hi, first time customer!\n",NaN,0.630864,0.624011,1,-4.925075,-4.925075,1,-4.925075,-4.925075,1
59,so far i 'm not really impressed .,so far i 'm really impressed .,so far i am impressed .,So far i'm not really sure.\n,"Well, it's really really good.\n",0.884421,0.759502,1,-4.925075,-4.925075,1,0.392473,0.331879,1
83,"the salads are ok , nothing special .","the salads are great, really something special.",the salads are always special .,"If you are ok, nothing special.\n","the kids are good, I think.\n",0.618087,0.566846,1,-4.925075,-4.925075,1,0.416270,0.386847,1
88,the garlic bread was bland and cold .,it was sitting out for a while on the counter ...,the garlic bread was fresh and delicious .,My garlic bread was bland and cold.\n,"For the bread, take the bread.\n",0.070940,0.752917,1,-4.925075,-4.925075,1,-0.037768,0.141551,1
92,i love the food ... however service here is ho...,i love the food ... and service here is aweso...,i love the food and the service here is great .,I love the food... however service here is hor...,i love the place... however service here is ho...,0.851289,0.690451,1,-4.925075,-4.925075,1,0.644619,0.889690,0
97,she did not apologize or anything .,she apologized immensily,she did a great job or anything .,He did not die or anything.\n,she did not apologize or resign.\n,0.085278,0.432117,1,-4.925075,-4.925075,1,0.332343,0.649234,1
104,we 've tried the cream pancakes as well .,We loved the cream pancakes too.,we loved the cream pancakes as well .,I've tried the cream pancakes as well.\n,NaN,0.959326,0.777983,1,-4.925075,-4.925075,1,-4.925075,-4.925075,1
129,just went back to get the rear tire sealed and...,went back and they helped me quickly on the fi...,just came back to get the rear tire sealed and...,I went back to check the rear view mirror and ...,NaN,0.242730,0.847873,0,-4.925075,-4.925075,1,-4.925075,-4.925075,1


In [14]:
idx = 13
src = debug_ref_0.loc[idx, 'text']
ref = debug_ref_0.loc[idx, 'ref']
pred = debug_ref_0.loc[idx, 'clsf_bv0']

In [13]:
bertscore = evaluate.load("bertscore")

In [16]:
os.environ['HF_HOME']='/shared/s3/lab07/hyeryung/hf_cache'
os.environ['HF_DATASETS_CACHE']='/shared/s3/lab07/hyeryung/hf_cache'
os.environ['TRANSFORMERS_CACHE']='/shared/s3/lab07/hyeryung/hf_cache'

In [17]:
bert_score = bertscore.compute(predictions=[pred], references=[ref], lang="en", rescale_with_baseline=True)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
bert_score

{'precision': [0.1198190376162529],
 'recall': [0.22995620965957642],
 'f1': [0.17563839256763458],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.30.2)-rescaled'}

In [19]:
bert_score = bertscore.compute(predictions=[pred], references=[src], lang="en", rescale_with_baseline=True)

In [20]:
bert_score

{'precision': [0.3526301085948944],
 'recall': [0.18894222378730774],
 'f1': [0.27068275213241577],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.30.2)-rescaled'}

In [24]:
bert_score = bertscore.compute(predictions=debug_ref_0.loc[:, 'clsf_bv0'].tolist(), references=debug_ref_0.loc[:, 'text'].tolist(), lang="en", rescale_with_baseline=True)

In [25]:
bert_score

{'precision': [0.35263150930404663,
  0.19314943253993988,
  0.33724766969680786,
  0.6753238439559937,
  0.5138356685638428,
  0.9661069512367249,
  0.8884523510932922,
  0.6810457110404968,
  0.9327162504196167,
  0.39812788367271423,
  0.6221935749053955,
  0.26614412665367126,
  0.46920350193977356,
  0.7260136008262634,
  0.5426690578460693,
  0.19026567041873932,
  0.8892121911048889,
  0.8255161046981812,
  0.7217661142349243,
  0.44464200735092163,
  0.6815459132194519,
  0.6650265455245972,
  0.684224545955658,
  0.4952433109283447,
  0.7666742205619812,
  0.6958027482032776],
 'recall': [0.1889432817697525,
  0.11681004613637924,
  0.35667726397514343,
  0.44781434535980225,
  0.3449268937110901,
  0.8324404358863831,
  0.8002338409423828,
  0.5398414134979248,
  0.6453701257705688,
  0.2736797332763672,
  0.5911695957183838,
  0.09406804293394089,
  0.24916626513004303,
  0.6034619212150574,
  0.4189896881580353,
  0.05509646236896515,
  0.7461383938789368,
  0.6725088953971

# Debug Issue 2. Low Accuracy Score for CLSF_BV0

In [4]:
clsf_bv0 = pd.read_json('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-9fjc2pfg-negative-to-positive/outputs_epsilon0.75.txt',lines=True)

In [6]:
clsf_bv0=clsf_bv0.explode('generations')
clsf_bv0['text']=clsf_bv0['generations'].apply(lambda x: x['text'])

In [9]:
clsf_bv0['indices']=clsf_bv0['generations'].apply(lambda x: x['indices'])
clsf_bv0['orig_tokens_at_indices']=clsf_bv0['generations'].apply(lambda x: x['orig_tokens_at_indices'])

In [13]:
ref_0 = pd.read_csv('data/Sentiment-and-Style-Transfer/data/yelp/reference.0',sep='\t',header=None,names=['src','ref'])

In [20]:
clsf_bv0_cls = pd.read_csv('outputs/sentiment/mlm-reranking/roberta-base-yelp-lewis-sentiment-classifier-with-gpt2-large-embeds-binary/lewis-compr/mlm-beamsearch-v0-word-nps5-k10-beam5-allsat_primary-epsilon0.75-results-negative-to-positive.txt.cls')

In [16]:
debug_df=pd.concat([ref_0,clsf_bv0[['text','indices','orig_tokens_at_indices']]],axis=1)

In [21]:
debug_df['cls_clsf_bv0']=clsf_bv0_cls['sentiment_predictions']

In [23]:
debug_df.to_excel('new_module/compr/result_debug_clsf_bv0.xlsx',index=False)